# 🗺️ Grid Path Search Lab
**SE 444: Introduction to AI - Prof. Anis Koubaa**

**Goal**: Implement and compare BFS, DFS, UCS, and A* algorithms for 2D grid navigation.

**Instructions**: 
- Run each cell step-by-step 
- Works in both Local Jupyter and Google Colab
- Experiment with different grids and start/goal positions

---


## 🔧 Setup
Import libraries and check environment.


In [ ]:
from collections import deque
import heapq
import time

# Check environment
try:
    import google.colab
    print("📍 Running in Google Colab")
except ImportError:
    print("📍 Running in Local Jupyter")

print("✅ Libraries imported successfully!")


## 🏗️ GridSearcher Class
Base class with helper functions for grid navigation.


In [ ]:
class GridSearcher:
    def __init__(self, grid):
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0]) if grid else 0
        self.nodes_explored = 0
        
    def reset_stats(self):
        self.nodes_explored = 0
    
    def is_valid(self, row, col):
        """Check if position is valid and passable."""
        return (0 <= row < self.rows and 0 <= col < self.cols and self.grid[row][col] == 0)
    
    def get_neighbors(self, pos):
        """Get valid neighboring positions."""
        row, col = pos
        neighbors = []
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:  # up, down, left, right
            new_row, new_col = row + dr, col + dc
            if self.is_valid(new_row, new_col):
                neighbors.append((new_row, new_col))
        return neighbors
    
    def manhattan_distance(self, pos1, pos2):
        """Calculate Manhattan distance heuristic."""
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    def reconstruct_path(self, parent, start, goal):
        """Reconstruct path using parent pointers."""
        path = []
        current = goal
        while current is not None:
            path.append(current)
            current = parent.get(current)
        path.reverse()
        return path
    
    def visualize(self, path=None):
        """Print grid with path visualization."""
        print("\\n📍 Grid: S=Start, G=Goal, █=Obstacle, .=Empty, *=Path")
        for row in range(self.rows):
            for col in range(self.cols):
                pos = (row, col)
                if self.grid[row][col] == 1:
                    print("█", end=" ")
                elif path and pos == path[0]:
                    print("S", end=" ")
                elif path and pos == path[-1]:
                    print("G", end=" ")
                elif path and pos in path:
                    print("*", end=" ")
                else:
                    print(".", end=" ")
            print()
        print()

print("✅ GridSearcher class defined!")


## 🔄 BFS - Breadth-First Search
**Strategy**: Explore level by level | **Structure**: Queue | **Optimal**: Yes


In [ ]:
def bfs(self, start, goal):
    """BFS: Level-by-level exploration using queue."""
    self.reset_stats()
    start_time = time.time()
    
    queue = deque([start])
    visited = {start}
    parent = {start: None}
    
    while queue:
        current = queue.popleft()  # FIFO
        self.nodes_explored += 1
        
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'BFS',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        for neighbor in self.get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                queue.append(neighbor)
    
    return None, {'algorithm': 'BFS', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add method to class
GridSearcher.bfs = bfs
print("✅ BFS implemented!")


## ⬇️ DFS - Depth-First Search
**Strategy**: Go deep, backtrack | **Structure**: Stack | **Optimal**: No


In [ ]:
def dfs(self, start, goal, max_depth=50):
    """DFS: Depth-first exploration using stack."""
    self.reset_stats()
    start_time = time.time()
    
    stack = [(start, 0)]  # (position, depth)
    visited = {start}
    parent = {start: None}
    
    while stack:
        current, depth = stack.pop()  # LIFO
        self.nodes_explored += 1
        
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'DFS',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        if depth < max_depth:
            for neighbor in reversed(self.get_neighbors(current)):
                if neighbor not in visited:
                    visited.add(neighbor)
                    parent[neighbor] = current
                    stack.append((neighbor, depth + 1))
    
    return None, {'algorithm': 'DFS', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add method to class
GridSearcher.dfs = dfs
print("✅ DFS implemented!")


## 💰 UCS - Uniform-Cost Search
**Strategy**: Lowest cost first | **Structure**: Priority Queue | **Optimal**: Yes


In [ ]:
def ucs(self, start, goal):
    """UCS: Uniform-cost search using priority queue."""
    self.reset_stats()
    start_time = time.time()
    
    pq = [(0, start)]  # (cost, position)
    costs = {start: 0}
    parent = {start: None}
    visited = set()
    
    while pq:
        current_cost, current = heapq.heappop(pq)
        
        if current in visited:
            continue
        visited.add(current)
        self.nodes_explored += 1
        
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'UCS',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        for neighbor in self.get_neighbors(current):
            new_cost = current_cost + 1  # Each step costs 1
            if neighbor not in costs or new_cost < costs[neighbor]:
                costs[neighbor] = new_cost
                parent[neighbor] = current
                heapq.heappush(pq, (new_cost, neighbor))
    
    return None, {'algorithm': 'UCS', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add method to class
GridSearcher.ucs = ucs
print("✅ UCS implemented!")


## ⭐ A* Search
**Strategy**: f(n) = g(n) + h(n) | **Heuristic**: Manhattan | **Optimal**: Yes


In [ ]:
def astar(self, start, goal):
    """A*: Heuristic-guided search using f(n) = g(n) + h(n)."""
    self.reset_stats()
    start_time = time.time()
    
    h_start = self.manhattan_distance(start, goal)
    pq = [(h_start, 0, start)]  # (f_cost, g_cost, position)
    g_costs = {start: 0}
    parent = {start: None}
    visited = set()
    
    while pq:
        f_cost, g_cost, current = heapq.heappop(pq)
        
        if current in visited:
            continue
        visited.add(current)
        self.nodes_explored += 1
        
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            return path, {
                'algorithm': 'A*',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time_ms': (time.time() - start_time) * 1000
            }
        
        for neighbor in self.get_neighbors(current):
            tentative_g = g_cost + 1
            if neighbor not in g_costs or tentative_g < g_costs[neighbor]:
                g_costs[neighbor] = tentative_g
                parent[neighbor] = current
                h_cost = self.manhattan_distance(neighbor, goal)
                f_cost = tentative_g + h_cost
                heapq.heappush(pq, (f_cost, tentative_g, neighbor))
    
    return None, {'algorithm': 'A*', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 
                  'time_ms': (time.time() - start_time) * 1000}

# Add method to class
GridSearcher.astar = astar
print("✅ A* implemented!")


## 🧪 Test & Compare Algorithms
Run all algorithms on test grids and compare performance.


In [ ]:
# Test grids
simple_grid = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]

maze_grid = [
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [1, 0, 0, 0, 0],
    [1, 0, 1, 1, 0],
    [0, 0, 0, 0, 0]
]

test_cases = [
    ("Simple 3x3", simple_grid, (0, 0), (2, 2)),
    ("Maze 5x5", maze_grid, (0, 0), (0, 4))
]

print("📋 Test grids defined!")


In [ ]:
def run_comparison(grid_name, grid, start, goal):
    """Run all algorithms and compare results."""
    print(f"\\n{'='*50}")
    print(f"🎯 Testing: {grid_name} | Start: {start} → Goal: {goal}")
    print(f"{'='*50}")
    
    searcher = GridSearcher(grid)
    searcher.visualize()
    
    algorithms = ['bfs', 'dfs', 'ucs', 'astar']
    results = []
    
    for algo in algorithms:
        try:
            path, stats = getattr(searcher, algo)(start, goal)
            results.append((algo.upper(), path, stats))
        except Exception as e:
            print(f"❌ {algo.upper()} failed: {e}")
    
    # Results table
    print(f"{'Algorithm':<8} {'Length':<8} {'Nodes':<8} {'Time(ms)':<10} {'Found'}")
    print("-" * 50)
    
    for algo, path, stats in results:
        found = "✅" if path else "❌"
        print(f"{algo:<8} {stats['path_length']:<8} {stats['nodes_explored']:<8} {stats['time_ms']:<10.2f} {found}")
    
    # Show best path
    for algo, path, stats in results:
        if path:
            print(f"\\n🏆 Solution by {algo}: {path}")
            searcher.visualize(path)
            break

# Run tests on both grids
for grid_name, grid, start, goal in test_cases:
    run_comparison(grid_name, grid, start, goal)


## 🎯 Create Your Own Test
Design a custom grid and test the algorithms. Modify the grid below and run to see results!


In [ ]:
# TODO: Create your own test grid (0=empty, 1=obstacle)
custom_grid = [
    [0, 0, 1, 0, 0],
    [0, 1, 1, 1, 0],
    [0, 0, 0, 0, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0]
]

# Set start and goal positions
start_pos = (0, 0)
goal_pos = (4, 4)

# Test your custom grid
run_comparison("Custom Grid", custom_grid, start_pos, goal_pos)


## 📊 Analysis Questions
Answer these based on your results:

1. **Which algorithm found the shortest path?** _____________
2. **Which explored the fewest nodes?** _____________
3. **How does the A* heuristic help?** _____________
4. **When might DFS be useful?** _____________

### Your Analysis:
*Write your observations here...*


## 🎓 Key Takeaways

✅ **BFS**: Guarantees shortest path, explores many nodes  
✅ **DFS**: Memory efficient, may find longer paths  
✅ **UCS**: Optimal for weighted graphs  
✅ **A***: Uses heuristics to be both optimal and efficient

### Real Applications:
- **GPS Navigation**: A* for route planning
- **Game AI**: Pathfinding for NPCs
- **Robotics**: Robot navigation
- **Network Routing**: Packet routing algorithms

**Next Step**: Try the 8-Puzzle lab for more complex state spaces!


# 🗺️ Grid Path Search Lab
**SE 444: Introduction to AI - Prof. Anis Koubaa**

Implement and compare BFS, DFS, UCS, and A* algorithms for 2D grid navigation.

---


## 🔧 Setup
Import libraries and check environment (works in both Jupyter and Colab).


In [ ]:
from collections import deque
import heapq
import time

# Check environment
try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

print(f"✅ Environment: {'Google Colab' if IN_COLAB else 'Local Jupyter'}")
print("✅ Libraries imported successfully!")


## 🏗️ GridSearcher Class
The base class with helper functions. **Your task**: Complete the TODO sections.


In [ ]:
class GridSearcher:
    """Class for performing search algorithms on 2D grids."""
    
    def __init__(self, grid):
        self.grid = grid
        self.rows = len(grid)
        self.cols = len(grid[0]) if grid else 0
        self.nodes_explored = 0
        
    def reset_stats(self):
        """Reset search statistics."""
        self.nodes_explored = 0
    
    def is_valid_position(self, row, col):
        """Check if position is valid and passable."""
        # TODO: Check bounds and obstacles
        return (0 <= row < self.rows and 
                0 <= col < self.cols and 
                self.grid[row][col] == 0)
    
    def get_neighbors(self, position):
        """Get all valid neighboring positions."""
        row, col = position
        neighbors = []
        # TODO: Generate neighbors in 4 directions
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # up, down, left, right
        for dr, dc in directions:
            new_row, new_col = row + dr, col + dc
            if self.is_valid_position(new_row, new_col):
                neighbors.append((new_row, new_col))
        return neighbors
    
    def manhattan_distance(self, pos1, pos2):
        """Calculate Manhattan distance heuristic."""
        # TODO: |x1-x2| + |y1-y2|
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])
    
    def reconstruct_path(self, parent, start, goal):
        """Reconstruct path using parent pointers."""
        path = []
        current = goal
        while current is not None:
            path.append(current)
            current = parent.get(current)
        path.reverse()
        return path
    
    def visualize_grid(self, path=None):
        """Print grid visualization."""
        print("\\nGrid: S=Start, G=Goal, █=Obstacle, .=Empty, *=Path")
        for row in range(self.rows):
            for col in range(self.cols):
                pos = (row, col)
                if self.grid[row][col] == 1:
                    print("█", end=" ")
                elif path and pos == path[0]:
                    print("S", end=" ")
                elif path and pos == path[-1]:
                    print("G", end=" ")
                elif path and pos in path:
                    print("*", end=" ")
                else:
                    print(".", end=" ")
            print()
        print()

print("✅ GridSearcher class defined!")


## 🔄 Breadth-First Search (BFS)
**Strategy**: Explore level by level  
**Data Structure**: Queue (FIFO)  
**Guarantee**: Shortest path


In [ ]:
def bfs(self, start, goal):
    """BFS implementation."""
    self.reset_stats()
    start_time = time.time()
    
    # TODO: Implement BFS
    queue = deque([start])
    visited = {start}
    parent = {start: None}
    
    while queue:
        current = queue.popleft()
        self.nodes_explored += 1
        
        if current == goal:
            path = self.reconstruct_path(parent, start, goal)
            stats = {
                'algorithm': 'BFS',
                'path_length': len(path) - 1,
                'nodes_explored': self.nodes_explored,
                'time': time.time() - start_time
            }
            return path, stats
        
        for neighbor in self.get_neighbors(current):
            if neighbor not in visited:
                visited.add(neighbor)
                parent[neighbor] = current
                queue.append(neighbor)
    
    return None, {'algorithm': 'BFS', 'path_length': -1, 
                  'nodes_explored': self.nodes_explored, 'time': time.time() - start_time}

# Add to class
GridSearcher.bfs = bfs
print("✅ BFS implemented!")


## ⬇️ Depth-First Search (DFS)
**Strategy**: Go deep, then backtrack  
**Data Structure**: Stack (LIFO)  
**Guarantee**: Finds path (not shortest)
